In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import random
import shutil

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

%matplotlib inline

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score
from scipy import stats

# scorefunc = roc_auc_score
scorefunc = average_precision_score

def get_auc_score(X, y, labelname, labeln, y_pred=None, model=None, foldpath=None, scorefunc=scorefunc):
    aucs = []
    foldsres = np.load(foldpath)[labelname][labeln][0]
    for train, valid, test in foldsres:
        train = np.concatenate((train, valid))
        Xtrain, Xtest = X[train,:], X[test,:]
        ytrain, ytest = y[train], y[test]
        if y_pred is not None:
            y_pred_test = y_pred[test]
            aucs.append(scorefunc(ytest, y_pred_test))
        elif model is not None:
            model.fit(Xtrain, ytrain)
            y_pred_test = model.predict_proba(Xtest)[:,1]
            aucs.append(scorefunc(ytest, y_pred_test))
#         print(roc_curve(ytest, y_pred_test))
    aucmean = np.mean(aucs)
    aucstd = np.std(aucs)
    dist = 1.96 * aucstd
    return aucmean, aucstd, [aucmean-dist, aucmean+dist] # http://biostats.bepress.com/cgi/viewcontent.cgi?article=1307&context=ucbbiostat

In [7]:
HRS = 48
WD = '../../Data/admdata_17f/%dhrs/non_series' % HRS
foldpath = '../../Data/admdata_17f/%dhrs/series/5-folds.npz' % HRS
foldcvpath = '../../Data/admdata_17f/%dhrs/series/cv/5-folds.npz' % HRS

mv_flag = np.array(np.load(os.path.join(WD, 'mv_flag.npy')).tolist())
sapsii = np.array(np.load(os.path.join(WD, 'sapsii.npz'))['sapsii'])
sofax = np.load(os.path.join(WD, 'sofa.npz'))['sofa'].reshape(-1, 1)
yall = np.genfromtxt(os.path.join(WD, 'output.csv'), delimiter=',')
sapsii_pred = sapsii[:, 4]
sapsii_score = sapsii[:, 5:].astype(np.float64)
sapsii_score[np.isnan(sapsii_score)] = 0.0

from sklearn.linear_model import LogisticRegression

for mp in [0,2,3,4,5]:
    y = yall[:, mp]
    sofam = LogisticRegression()
    print(mp)
    # cv sapsii
    cv_sapsii_res = get_auc_score(sapsii_score[~mv_flag], y[~mv_flag], labelname='folds_ep_mor', labeln=mp, y_pred=sapsii_pred[~mv_flag], model=None, foldpath=foldcvpath)
    # cv sapsii refitted
    cv_sapsii_refitted_res = get_auc_score(sapsii_score[~mv_flag], y[~mv_flag], labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldcvpath)
    # cv sofa
    cv_sofa_res = get_auc_score(sofax[~mv_flag].reshape(-1,1), y[~mv_flag], labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldcvpath)
    # # mv sapsii
    # print('mv sapsii:', get_auc_score(sapsii_score[mv_flag], y[mv_flag], 10, y_pred=sapsii_pred[mv_flag], model=None))
    # # mv sofa
    # print('mv sofa:', get_auc_score(sofax[mv_flag].reshape(-1,1), y[mv_flag], 10, y_pred=None, model=LogisticRegression()))
    # # mv sapsii refitted
    # print('mv sapsii refitted:', get_auc_score(sapsii_score[mv_flag], y[mv_flag], 10, y_pred=None, model=LogisticRegression()))
    # all sapsii
    all_sapsii_res = get_auc_score(sapsii_score, y, labelname='folds_ep_mor', labeln=mp, y_pred=sapsii_pred, model=None, foldpath=foldpath)
    # all sapsii refitted
    all_sapsii_refitted_res = get_auc_score(sapsii_score, y, labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldpath)
    # all sofa
    all_sofa_res = get_auc_score(sofax.reshape(-1,1), y, labelname='folds_ep_mor', labeln=mp, y_pred=None, model=LogisticRegression(), foldpath=foldpath)
    for t in [all_sapsii_res, all_sapsii_refitted_res, all_sofa_res, cv_sapsii_res, cv_sapsii_refitted_res, cv_sofa_res]:
        print(t[0], t[1], t[2])
    print('\n\n')

0
0.337253187878 0.0141062270055 [0.3096049829473555, 0.36490139280872824]
0.382287972382 0.0119101363115 [0.35894410521120745, 0.40563183955234527]
0.285203375656 0.0166588829012 [0.25255196516933676, 0.31785478614206342]
0.344764184729 0.0218634430793 [0.3019118362936612, 0.38761653316443517]
0.387335790291 0.016335924444 [0.35531737838084371, 0.41935420220147496]
0.299606146697 0.0199214866546 [0.2605600328542475, 0.33865226054019354]



2
0.0320093891458 0.0182593007607 [-0.0037788403451989097, 0.067797618636744272]
0.0688547267418 0.0301407609658 [0.009778835248918108, 0.12793061823473756]
0.0120158650931 0.00403500875016 [0.0041072479427466616, 0.019924482243386117]
0.0339239935245 0.0342945215603 [-0.033293268733723728, 0.10114125578265082]
0.0214946698183 0.0137632879142 [-0.0054813744934384713, 0.048470714130049136]
0.00609384559675 0.00417460864394 [-0.0020883873453780922, 0.014276078538876181]



3
0.108211835256 0.0150314182488 [0.078750255488773652, 0.13767341502421923]
0.

In [4]:
print(sapsii)

[[58526 100001 275225 ..., 0 0 6]
 [9895 100006 291788 ..., 0 10 6]
 [23018 100007 217937 ..., 0 0 8]
 ..., 
 [19412 199995 255092 ..., 0 0 8]
 [27200 199998 293589 ..., 0 0 8]
 [40370 199999 211153 ..., 0 0 6]]
